In [2]:
%cd ~/bi_practice1/pro1//binfo1-work/

/storage/home/tdkcha/bi_practice1/pro1/binfo1-work


In [ ]:
# BAM -> pileup
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup
# filtering (read counts)
!cat CLIP-35L33G.pileup | awk '$4 > 50{print$0}' > filtered_CLIP-35L33G.pileup

In [ ]:
import pandas as pd
import re
import numpy as np
from Bio import SeqIO
from collections import Counter

# Define patterns to be removed
patterns = {
    "common": re.compile('[<>$*#\-\+\d]'),
    "start_of_read": re.compile('\^.'),
    "lower_bases": re.compile('[acgtnN]'),
    "upper_bases": re.compile('[ACGTnN]')
}

In [ ]:
# Calculate entropy
def cal_SE(dna_sequence):
    bases, counts = np.unique(list(dna_sequence), return_counts=True)
    probabilities = counts / counts.sum()
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value

In [ ]:
# Read the file
filtered_data = pd.read_csv('CLIP-35L33G_filtered.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])

# Filter based on chromosome name
filtered_data = filtered_data[filtered_data['chrom'].str.startswith("chr")].copy()

# Remove unwanted characters
filtered_data['matches'] = filtered_data['basereads'].str.replace(patterns["common"], '').str.replace(patterns["start_of_read"], '')

In [ ]:
# Duplicate matches column to differentiate between plus and minus strands
filtered_data = filtered_data.assign(plus_strand=filtered_data['matches'], minus_strand=filtered_data['matches'])

# Remove bases in each template
filtered_data['plus_strand'] = filtered_data['plus_strand'].str.replace(patterns["lower_bases"], '')
filtered_data['minus_strand'] = filtered_data['minus_strand'].str.replace(patterns["upper_bases"], '')

In [ ]:
# Calculate entropy
filtered_data['plus_entropy'] = filtered_data['plus_strand'].map(cal_SE)
filtered_data['minus_entropy'] = filtered_data['minus_strand'].map(cal_SE)

# Apply filtering
p_fil_stand = (filtered_data['plus_strand'].str.len() > 50) & (filtered_data['plus_entropy'] > 0.8)
m_fil_stand = (filtered_data['minus_strand'].str.len() > 50) & (filtered_data['minus_entropy'] > 0.8)

In [ ]:
SE_p = filtered_data[p_fil_stand][['chrom','pos']]
SE_p = SE_p[SE_p['chrom'] != "chrM"]

SE_m = filtered_data[m_fil_stand][['chrom','pos']]
SE_m = SE_m[SE_m['chrom'] != "chrM"]

In [21]:
%cd ~/bi_practice1/pro1//binfo1-work/Ref
!gunzip *.fa.gz
!cat *.fa > mm39.fa

/storage/home/tdkcha/bi_practice1/pro1/binfo1-work/Ref


In [34]:
%cd ~/bi_practice1/pro1//binfo1-work/

/storage/home/tdkcha/bi_practice1/pro1/binfo1-work


In [ ]:
ref_gen = SeqIO.to_dict(SeqIO.parse("/Ref/mm39.fa", "fasta"))

In [ ]:
p_motifs = []
m_motifs = []

for _, row in SE_p.iterrows():
    full_seq = str(ref_gen[row['chrom']].seq[(row['pos']-11):(row['pos']+10)]) 
    hexamer = full_seq[8:14] # Extract hexamer
    p_motifs.append(hexamer)

for _, row in SE_m.iterrows():
    full_seq = str(ref_gen[row['chrom']].seq[(row['pos']-11):(row['pos']+10)].reverse_complement())
    hexamer = full_seq[8:14] # Extract hexamer
    m_motifs.append(hexamer)

In [ ]:
motifs = p_motifs + m_motifs
motif_counter = Counter(motifs)

sorted_motif_counter = sorted(motif_counter.items(), key=lambda x: x[1], reverse=True)

with open("/WebLogo_Input.txt", 'w') as f:
    for item in sorted_motif_counter:
        for _ in range(item[1]):  
            f.write(f"{item[0]}\n")

In [ ]:
###Base probability###
p_motifs = []
m_motifs = []

# 추출할 시퀀스의 길이는 -10부터 +10까지의 전체 21개 위치이므로, 인덱스 범위를 조정
for _, row in SE_p.iterrows():
    full_seq = str(ref_gen[row['chrom']].seq[(row['pos']-10):(row['pos']+10)])
    p_motifs.append(full_seq)

for _, row in SE_m.iterrows():
    full_seq = str(ref_gen[row['chrom']].seq[(row['pos']-10):(row['pos']+10)].reverse_complement())
    m_motifs.append(full_seq)

p_motifs = [motif.replace('T', 'U') for motif in p_motifs]
m_motifs = [motif.replace('T', 'U') for motif in m_motifs]

In [ ]:
# 플러스 및 마이너스 시퀀스를 합쳐 전체 모티프 리스트 만듬
total_motifs = p_motifs + m_motifs

# 합친 시퀀스를 텍스트 파일에 저장
with open( "/WebLogo_FullMotifs.txt", 'w') as f:
    for item in total_motifs:
        f.write("%s\n" % item)